# Run stock-price prediction model

## Notebook ([`papermill`](https://papermill.readthedocs.io/en/latest/index.html)) parameters

In [1]:
run_time = '2019-09-01T10:00'
stop_minutes = 10
dir = None
earliest_data_cutoff = '2019-04-01'
ticker = 'AAPL'
batch_size = 100000
max_epochs = 10000

## Date/Time helpers

In [2]:
from sys import executable as python
from datetime import datetime as dt, date, time, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
now = dt.now()
today = now.date()

if run_time:
    run_time = parse(run_time)
else:
    run_time = dt.now()

run_date = run_time.date()

if stop_minutes:
    stop_at = run_time + Δ(minutes=stop_minutes)
else:
    stop_at = None

earliest_data_cutoff = parse(earliest_data_cutoff)
earliest_data_cutoff_date = earliest_data_cutoff.date()
    
print('Running at %s (stopping within %dmins)' % (run_time.strftime('%Y-%m-%d %H:%M:%S'), stop_minutes))

Running at 2019-09-01 10:00:00 (stopping within 10mins)


## Initialize `data`, `models` directories

In [3]:
from pathlib import Path
if not dir:
    dir = Path.cwd()

data_dir = dir / 'data'
data_dir.mkdir(parents=True, exist_ok=True)
models_dir = dir / 'models'

## Read IEX API credentials
Search for a secret key in:
- `$IEX_SECRET_KEY` env var
- `iex.secret_key` in a config file located at:
  - `$CONFIG_PATH` if set
  - `~/.config/iex.ini` otherwise

In [4]:
from os import environ as env

api = 'https://cloud.iexapis.com'

SECRET_KEY_VAR = 'IEX_SECRET_KEY'
CONFIG_PATH_VAR = 'CONFIG_PATH'

if SECRET_KEY_VAR in env:
    secret_key = env[SECRET_KEY_VAR]
else:
    if CONFIG_PATH_VAR in env:
        config_path = Path(env[CONFIG_PATH_VAR])
    else:
        config_path = Path.home() / '.config' / 'iex.ini'

    if not config_path.exists():
        raise Exception("No %s env var, and config path %s doesn't exist" % (SECRET_KEY_VAR, config_path))

    from configparser import ConfigParser
    config = ConfigParser()
    config.read(str(config_path))
    iex_config = config['iex']
    secret_key = iex_config['secret_key']

## Initialize `dates` and `minutes` that trading occurs during

In [5]:
start_minute = time(9, 30)
end_minute = time(16, 0)
def get_minutes():
    minute = start_minute
    while minute < end_minute:
        yield minute
        hr = minute.hour
        min = minute.minute
        min += 1
        if min == 60:
            min = 0
            hr += 1
        minute = time(hr, min)

minutes = list(get_minutes())
num_minutes = len(minutes); num_minutes

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        # only emit weekdays
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

start_date = earliest_data_cutoff_date
end_date = run_date + Δ(days=1)
dates = list(get_dates(start_date, end_date))
num_dates = len(dates)

print(
    'Fetching/Processing data from %d days ([%s,%s)), %d minutes per day' % (
        num_dates, 
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d'),
        num_minutes,
    )
)

Fetching/Processing data from 110 days ([2019-04-01,2019-09-02)), 390 minutes per day


In [6]:
def get_ticker_dir(ticker):
    return data_dir / ticker

def get_ticker_date_path(ticker, date):
    return get_ticker_dir(ticker) / date.strftime('%Y%m%d')

In [7]:
!{python} -m pip install -Uq requests
from requests import get as GET

In [8]:
import json

def fetch(date, ticker, refetch_partial=False):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    refetch = False
    prev_data = None
    if out_path.exists():
        if refetch_partial:
            with out_path.open('r') as f:
                prev_data = json.load(f)
                if len(prev_data) < num_minutes:
                    refetch = True
                    print(
                        'Re-fetching data for %s from %s (found %d per-minute quotes instead of %d)' % (
                            ticker, 
                            date_str, 
                            len(prev_data), 
                            num_minutes
                        )
                    )
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    if prev_data is None or len(data) > len(prev_data):
        with out_path.open('wb') as f:
            f.write(resp.content)

        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True
    elif len(data) < len(prev_data):
        raise Exception('Found %d data, less than previous amount %d' % (len(data), len(prev_data)))
    else:
        print('Re-fetched %s, found same %d data' % (out_path, len(data)))

    return False

In [9]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

Fetch quotes between a start point in the past (IEX seems to serve 6-7mos of historic data) and today

In [10]:
%%time

N = 32  # fetch parallelism
refetch_empty = False

Parallel(n_jobs=N)(
    delayed(fetch)(date, ticker, refetch_partial=refetch_empty or date == today)
    for date in dates 
); None

CPU times: user 517 ms, sys: 223 ms, total: 740 ms
Wall time: 7.38 s


In [11]:
!{python} -m pip install -Uq pandas matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    exp, log, sqrt
from numpy.random import shuffle, permutation
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd
from scipy.stats import describe

These are the columns that we receive from IEX, describing trades that took place there:

In [12]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]

In [13]:
def load_ticker_date_df(date, ticker, latest_data_datetime=None):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    # Convert "date", "minute" columns into a single "datetime" column
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    df.drop(columns=['date', 'minute'])
    if latest_data_datetime:
        df = df.loc[df.datetime <= latest_data_datetime]
    
    # Drop other columns ("market"-data columns, which are updated on a 15-min delay)
    df = df[['datetime'] + features]

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    return df

In [14]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    elif type(limit) == int:
        ds = dates[:limit]
    elif type(limit) == date:
        ds = [ date for date in dates if date < limit ]        
    elif type(limit) == dt:
        ds = [ date for date in dates if date < limit.date() ]
    else:
        raise Exception('Unrecognized limit: %s' % limit)

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker, latest_data_datetime=run_time) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    for col in features:
        # "quantity" features (volume, numberOfTrades, notional) are sometimes -1 instead of NaN
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [15]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 3.08 s, sys: 51.8 ms, total: 3.13 s
Wall time: 3.18 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.650,191.645,190.600,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.980,190.980,190.640,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.930,191.090,190.780,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.830,191.010,190.760,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.700,190.760,190.600,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-08-30 15:55:00,207.690,208.460,208.550,207.690,208.350,18889,3935525.955,212
2019-08-30 15:56:00,208.450,208.310,208.450,208.250,208.328,7071,1473088.595,75
2019-08-30 15:57:00,208.290,208.220,208.380,208.220,208.271,26268,5470873.575,143


## Fill missing data
Minutes where no trades were recorded can be assigned prices based on previous minutes' closing prices

We don't attempt to cross day boundaries, as the moves overnight are often much larger than minute-to-minute moves during the day, and skew the data.

Make a series representing minutes that are the first of their day (i.e. 9:30am)

In [16]:
prev_datetime = aapl.index.to_series().shift(1)
datetime = aapl.index.to_series()

def to_date(dt):
    return dt.date()

day_start = datetime.apply(to_date) != prev_datetime.apply(to_date)

Populate each minute with the previous minute's closing price

Assume each day's initial opening is the same as an imagined closing price from a minute prior (when trading wasn't open).

In [17]:
prev_close = aapl.close.shift(1)
prev_close[day_start] = aapl.open
aapl['prev_close'] = prev_close

Track minutes where no trades occurred:

In [18]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725
...,...,...,...,...,...,...,...,...,...
2019-08-29 12:43:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.770
2019-08-29 12:47:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.780
2019-08-29 14:01:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.945


Spot-check: consecutive pairs of minutes where no trades occurred

In [19]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False))
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-07-24 13:00:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.12
2019-07-24 13:01:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-07-24 13:24:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.90
2019-07-24 13:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-08-23 14:16:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,202.66
2019-08-23 14:17:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-08-27 13:04:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,204.02
2019-08-27 13:05:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-08-27 13:10:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,204.11


Group by date, and forward-fill "previous closing" price.

For minutes when no trades occurred, we will treat the last valid closing price as both the opening and closing price. However, we don't persist these across day boundaries (this results in apparent large jumps at opening that skew training).

In [20]:
grouped = aapl.groupby(to_date)
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill'))
aapl.count()

open              39092
close             39092
high              39092
low               39092
average           39092
volume            39207
notional          39207
numberOfTrades    39207
prev_close        39207
dtype: int64

Fill `NaN` openings with the previous valid closing price, which is interpreted as the start and end price for a minute where no activity occurred:

In [21]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)
aapl.count()

open              39207
close             39207
high              39207
low               39207
average           39207
volume            39207
notional          39207
numberOfTrades    39207
prev_close        39207
dtype: int64

We expect that there are no remaining `NaN`s in the dataset (unless they appear during the first minute of the day; at the time of writing that has not been observed)

Spot check that `NaN` entries have been filled in based on the previous minute's closing price:

In [22]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.050,192.990,193.014,1100,212316.000,11,192.990
2019-04-02 13:29:00,193.000,193.000,193.000,193.000,193.000,0,0.000,0,193.000
2019-04-05 13:35:00,196.535,196.540,196.540,196.535,196.537,205,40290.200,3,196.495
2019-04-05 13:36:00,196.540,196.540,196.540,196.540,196.540,0,0.000,0,196.540
2019-04-05 14:28:00,196.820,196.795,196.820,196.795,196.811,731,143868.985,13,196.820
...,...,...,...,...,...,...,...,...,...
2019-08-29 14:01:00,208.945,208.945,208.945,208.945,208.945,0,0.000,0,208.945
2019-08-29 14:52:00,208.550,208.470,208.570,208.470,208.524,288,60054.880,9,208.500
2019-08-29 14:53:00,208.470,208.470,208.470,208.470,208.470,0,0.000,0,208.470


## Log-fold transforms

Market-data time-series are frequently approximated as being log-normally distributed; the fold-change from one interval to the next is considered to be normally distributed.

Here we copy the price data above, make `open` a log-fold change over the previous `close`, and express other intra-minute features as a log-fold change over their corresponding `open`ing price:

In [23]:
lg = aapl.copy()
for col in fill_cols[1:]:
    lg[col] = log(lg[col] / lg.open)
lg.open = log(lg.open / lg.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][l_or(nan_idxs, nan_idxs.shift(1).fillna(False))]

,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-02 13:30:00,0.000414,0.000000,0.000362,-0.000052,0.000104
2019-04-05 13:36:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 13:37:00,-0.000102,-0.000051,0.000000,-0.000051,-0.000010
2019-04-05 14:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
2019-08-29 14:02:00,-0.000574,0.000168,0.000168,-0.000024,-0.000019
2019-08-29 14:53:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-08-29 14:54:00,0.000240,-0.000048,0.000048,-0.000096,-0.000005


In [24]:
lg.count()

open              39207
close             39207
high              39207
low               39207
average           39207
volume            39207
notional          39207
numberOfTrades    39207
dtype: int64

Normalize and rename some features:
- `trades` ⟶ `sqrt(numberOfTrades)` (variance-stabilizing transform for a Poisson distribution)
- `volume` ⟶ `avgVol` (divide out then number of trades, and `log`-transform to attempt to normalize outliers

In [25]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735
...,...,...,...,...,...,...,...
2019-08-30 15:55:00,0.000024,0.003701,0.004132,0.000000,0.003173,14.560220,4.500910
2019-08-30 15:56:00,-0.000048,-0.000672,0.000000,-0.000960,-0.000585,8.660254,4.556820
2019-08-30 15:57:00,-0.000096,-0.000336,0.000432,-0.000336,-0.000091,11.958261,5.218691


Add `next_close` to each minute, which is the log-fold change of the next-minute's closing price over the current minute's.

This will be treated as the "label" for a training sample anchored at each minute: given the current minute's activity (and that of several minutes prior), can we predict the next minute's closing price?

The next minute's opening- and intra-minute features (high, low, average) are not trained on / learned; IEX publishes each minute's data about halfway through to the next minute, so predicting the next minute's closing price is a good goal for the purposes of this example.

In [26]:
grouped = lg.groupby(to_date)

def get_next_close(df):
    # the next minute's closing price is a combination of:
    # - the next minute's "open" field (log-fold change over current-minute close)
    # - the next minute's "close" field (log-fold change over next-minute open)
    df['next_close'] = df.open.shift(-1) + df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955,0.001729
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253,-0.000262
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048,-0.000524
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817,-0.000681
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735,0.000786
...,...,...,...,...,...,...,...,...
2019-08-30 15:55:00,0.000024,0.003701,0.004132,0.000000,0.003173,14.560220,4.500910,-0.000720
2019-08-30 15:56:00,-0.000048,-0.000672,0.000000,-0.000960,-0.000585,8.660254,4.556820,-0.000432
2019-08-30 15:57:00,-0.000096,-0.000336,0.000432,-0.000336,-0.000091,11.958261,5.218691,0.000096


`next_close` was populated within each day above; that means the final minute of each day is missing a label, so we drop them:

In [27]:
lg.dropna(how='any', inplace=True)
lg.count()

open          39106
close         39106
hi            39106
lo            39106
avg           39106
trades        39106
avgVol        39106
next_close    39106
dtype: int64

## Persistent assignment of labeled samples to {training, validation} sets
Assigning samples to validation vs training should be persistent; we will train iteratively over time on the full history up to that point, and samples marked for validation should always be held out from training.

We achieve this by seeding a PRNG for each sample based on that sample's minute ID (`int(strftime('%Y%m%d%H%M'))`), and pulling a single random float. The set of samples whose floats are less than $V$ should represent $100V$% of the data, and can be used collectively as a validation-split representing proportion $V$ of the total set of labeled samples.

In [28]:
from random import random, seed
validation_split = 0.2

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_rand(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random()

datetime = lg.index.to_series()  # update index since we dropped the last minute of each day
vrs = datetime.apply(get_validation_rand).rename('validation_rand')
vfs = (vrs < validation_split).rename('validation?')
tn, vn = vfs[~vfs].count(), vfs[vfs].count()
n = tn + vn
tf, vf = tn / n, vn / n
print('%d training samples (%.2f%%), %d validation (%.2f%%)' % (tn, 100 * tf, vn, 100 * vf))

31177 training samples (79.72%), 7929 validation (20.28%)


Separate the log-transformed data into training and validation sets.

Also compute {$\mu$, $\sigma$} of the training sets, which will be used to standardize the data.

In [29]:
lg_t = lg[~vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

,μ,σ
open,0.000009,0.000211
close,-0.000010,0.000617
hi,0.000306,0.000452
lo,-0.000322,0.000485
avg,-0.000002,0.000388
trades,4.092010,1.754347
avgVol,4.477675,0.426784
next_close,-0.000002,0.000642


Helpers for showing descriptive statistics of each column of a DataFrame:

In [30]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [31]:
desc(lg)

,n,μ,σ,skewness,kurtosis,min,max
open,39106,9.408198e-06,0.000211,-0.155289,7.292357,-0.002911,0.002457
close,39106,-7.737857e-06,0.000611,-0.145093,47.500531,-0.018992,0.014990
hi,39106,3.062325e-04,0.000453,4.843083,60.838604,0.000000,0.014990
lo,39106,-3.207058e-04,0.000476,-7.148552,193.132120,-0.023854,0.000000
avg,39106,-1.450797e-06,0.000383,-1.234469,79.807202,-0.015078,0.007264
trades,39106,4.091723e+00,1.757447,1.258400,3.531386,0.000000,21.679483
avgVol,39106,4.477167e+00,0.427511,-4.069562,43.707113,0.000000,7.509961
next_close,39106,-6.104338e-07,0.000639,-0.352563,40.979878,-0.018822,0.015714


The log-transform has captures the idea that, intuitively, a price is as likely to double as half. 

However, it's worth noting that the high kurtosis values imply that the data is not perfectly *normally distributed* after the log-transform; price movements are empirically "heavy-tailed", as we see here.

## Standardize data
Subtract and divide out the mean and stddev of the training features.

Note that:
- the training {$\mu$,$\sigma$} will be used for validation samples as well
- we store $-\mu/\sigma$ (pre-standardization $0$) in `zeros`, for use padding sliding windows later

In [32]:
normed = lg.copy()
zeros = -means / stddevs; zeros

for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,-0.044929,-8.426723,-0.676385,-10.612304,-6.139279,1.448539,0.279484,2.698222
2019-04-01 09:31:00,1.196131,2.394970,2.567613,0.015686,0.831420,0.891981,0.355162,-0.405030
2019-04-01 09:32:00,1.937181,-1.088419,-0.329241,-2.360083,-1.465836,0.789591,-0.341220,-0.813413
2019-04-01 09:33:00,1.194311,-1.258847,-0.329096,-1.712537,-0.452967,0.629375,-0.109794,-1.058987
2019-04-01 09:34:00,-1.781320,-0.494732,-0.676385,-1.065921,-1.265189,-0.357916,0.054032,1.228375
...,...,...,...,...,...,...,...,...
2019-08-30 15:54:00,-0.957272,1.617088,1.879703,0.664705,2.094389,2.636758,-0.056094,5.807673
2019-08-30 15:55:00,0.069011,6.017106,8.460058,0.664705,8.191390,5.967013,0.054441,-1.118760
2019-08-30 15:56:00,-0.271969,-1.074153,-0.676385,-1.315111,-1.503964,2.603958,0.185444,-0.670431


In [33]:
desc(normed)

,n,μ,σ,skewness,kurtosis,min,max
open,39106,-0.000402,0.997163,-0.155289,7.292357,-13.823051,11.584105
close,39106,0.002908,0.991498,-0.145093,47.500531,-30.785146,24.325739
hi,39106,0.000700,1.002105,4.843083,60.838604,-0.676385,32.465918
lo,39106,0.003258,0.981648,-7.148552,193.132120,-48.534242,0.664705
avg,39106,0.002606,0.988423,-1.234469,79.807202,-38.891413,18.746506
trades,39106,-0.000163,1.001767,1.258400,3.531386,-2.332498,10.025085
avgVol,39106,-0.001191,1.001703,-4.069562,43.707113,-10.491668,7.104967
next_close,39106,0.002081,0.996419,-0.352563,40.979878,-29.329037,24.491688


We continue to observe the presence of heavy outliers (≈10s of standard deviations, as evidenced by the `min`/`max` values in many columns). 

Dropping outliers seems unwise (sudden large price movements are arguably most important to be aware of!), so we'll proceed without further corrections.

Separate the `next_close` ("label") column, and name it `y`:

In [34]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,-0.044929,-8.426723,-0.676385,-10.612304,-6.139279,1.448539,0.279484
2019-04-01 09:31:00,1.196131,2.394970,2.567613,0.015686,0.831420,0.891981,0.355162
2019-04-01 09:32:00,1.937181,-1.088419,-0.329241,-2.360083,-1.465836,0.789591,-0.341220
2019-04-01 09:33:00,1.194311,-1.258847,-0.329096,-1.712537,-0.452967,0.629375,-0.109794
2019-04-01 09:34:00,-1.781320,-0.494732,-0.676385,-1.065921,-1.265189,-0.357916,0.054032
...,...,...,...,...,...,...,...
2019-08-30 15:54:00,-0.957272,1.617088,1.879703,0.664705,2.094389,2.636758,-0.056094
2019-08-30 15:55:00,0.069011,6.017106,8.460058,0.664705,8.191390,5.967013,0.054441
2019-08-30 15:56:00,-0.271969,-1.074153,-0.676385,-1.315111,-1.503964,2.603958,0.185444


## Rolling window, numpy `ndarray` conversion
Create a numpy array from a rolling 30-minute window of the standardized features:

In [35]:
window = 30

cols = normed.columns
x = np.moveaxis(
    array([ 
        concat(
            [ 
                normed[col].shift(i)
                for i in reversed(range(window)) 
            ], 
            axis=1
        ) \
        .fillna(zeros[col]) \
        .values
        for col in cols
    ]),
    0, 2
)
x.shape    

(39106, 30, 7)

For each minute, we have the last 30-minutes of data about the 7 features.

Partial windows are padded with "0"s (pre-standardization 0; actually $-\mu/\sigma$ in post-standardization space, taken from the `zeros` array above)

Store training/validation data, and assign the full `x` and `y` datasets as a concatenation of training and validation samples, respectively.

In [36]:
tx = x[~vfs]
vx = x[ vfs]
ty = y[~vfs].to_numpy()
vy = y[ vfs].to_numpy()
[ a.shape for a in [ tx,ty, vx,vy, x,y ] ]

[(31177, 30, 7), (31177,), (7929, 30, 7), (7929,), (39106, 30, 7), (39106,)]

In [37]:
factor = exp(y * stddevs.next_close + means.next_close).rename('factor')
next_close = (aapl.close * factor).rename('next_close')
concat([ aapl, next_close, factor ], axis=1)

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,next_close,factor
datetime,,,,,,,,,,,
2019-04-01 09:30:00,191.645,190.650,191.645,190.600,191.189,4320,825935.940,44,191.645,190.980,1.001731
2019-04-01 09:31:00,190.700,190.980,190.980,190.640,190.761,3246,619210.510,32,190.650,190.930,0.999738
2019-04-01 09:32:00,191.060,190.930,191.090,190.780,190.951,2253,430211.740,30,190.980,190.830,0.999476
2019-04-01 09:33:00,190.980,190.830,191.010,190.760,190.946,2241,427911.290,27,190.930,190.700,0.999319
2019-04-01 09:34:00,190.760,190.700,190.760,190.600,190.666,1069,203822.465,12,190.830,190.850,1.000787
...,...,...,...,...,...,...,...,...,...,...,...
2019-08-30 15:55:00,207.690,208.460,208.550,207.690,208.350,18889,3935525.955,212,207.685,208.310,0.999280
2019-08-30 15:56:00,208.450,208.310,208.450,208.250,208.328,7071,1473088.595,75,208.460,208.220,0.999568
2019-08-30 15:57:00,208.290,208.220,208.380,208.220,208.271,26268,5470873.575,143,208.310,208.240,1.000096


## Build model

In [38]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [39]:
def create_model():
    model = Sequential()
    model.add(SimpleRNN(4, input_shape=(window, len(cols))))
    model.add(Dense(1))
    model.build()
    model.compile(loss='mae', optimizer='adam')
    return model

In [41]:
import re
regex = '^\d{4}-\d\d-\d\dT\d\d:\d\d$'
ckpt_dirs = sorted([
    dir
    for dir in models_dir.iterdir() 
    if re.match(regex, dir.name)
])
model = create_model()
if ckpt_dirs:
    ckpt_dir = ckpt_dirs[-1]
    print('Loading model from %s' % ckpt_dir)
    ckpt = tf.train.latest_checkpoint(str(ckpt_dir))
    print('Found model %s' % ckpt)
    model.load_weights(ckpt)
else:
    print('No pre-existing models found; using untrained model')

model.summary()

Loading model from /Users/ryan/c/pipelines/notebooks/models/2019-08-01T10:00
Found model /Users/ryan/c/pipelines/notebooks/models/2019-08-01T10:00/2019-08-01T10:00_1660.ckpt
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 48        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


As a baseline for evaluating our model's training progress, compute the mean absolute error we'd expect from a network that simply learned to predict the mean of the training labels for every sample:

In [42]:
mean(abs(ty - mean(ty))), mean(abs(vy - mean(vy)))

(0.646456627234889, 0.6375417335494202)

In [43]:
def diffs_df(model):
    def stats(x, y, name=None, predict=True):
        suffix = ' (%s)' % name if name else ''
        if predict:
            predx = model.predict(x)[:, 0]
        else:
            predx = x

        diffs = (y - predx)
        mae = abs(diffs)
        mse = mae**2

        return DF({
            ('y^' + suffix): predx,
            ('Δ' + suffix): diffs,
            ('|Δ|' + suffix): mae,
            ('Δ²' + suffix): mse,
        })

    train = stats(tx, ty, 't')
    val = stats(vx, vy, 'v')
    predx_t = train['y^ (t)']
    predx_v = val['y^ (v)']
    predx = concat([ predx_t, predx_v ])
    all = stats(predx, np.concatenate([ ty, vy ]), name='*', predict=False)

    return concat([
        desc(train),
        desc(val),
        desc(all)
    ])

In [44]:
diffs_df(model)

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),31177,0.019310,0.044043,0.966544,2.390782,-3.389530e-01,0.397504
Δ (t),31177,-0.019310,0.999608,-0.458924,47.647831,-2.954934e+01,24.384961
|Δ| (t),31177,0.646016,0.763046,5.959390,121.282920,1.600787e-05,29.549341
Δ² (t),31177,0.999556,7.043102,84.318063,9277.115890,2.562518e-10,873.163563
y^ (v),7929,0.018354,0.043605,0.981439,1.974194,-1.180170e-01,0.334107
Δ (v),7929,-0.008089,0.982866,-0.257578,15.105335,-1.208585e+01,12.874183
|Δ| (v),7929,0.636873,0.748621,3.974260,32.428902,1.239002e-05,12.874183
Δ² (v),7929,0.965970,3.996119,21.236059,685.410003,1.535126e-10,165.744583
y^ (*),39106,0.019116,0.043956,0.969615,2.309177,-3.389530e-01,0.397504
Δ (*),39106,-0.017035,0.996234,-0.419928,41.406780,-2.954934e+01,24.384961


In [45]:
model_ckpt_path = models_dir / run_time.strftime('%Y-%m-%dT%H:%M') / 'cp-{epoch:04d}.ckpt'; model_ckpt_path

PosixPath('/Users/ryan/c/pipelines/notebooks/models/2019-09-01T10:00/cp-{epoch:04d}.ckpt')

In [46]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[
              EarlyStopping(patience=100, restore_best_weights=True),
              ModelCheckpoint(str(model_ckpt_path), save_weights_only=True, save_best_only=True, period=20),
          ]
         )
diffs_df(model)

Train on 31177 samples, validate on 7929 samples
Epoch 1/10000
31177/31177 [==============================] - 1s 44us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 2/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 3/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 4/10000
31177/31177 [==============================] - 0s 7us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 5/10000
31177/31177 [==============================] - 0s 7us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 6/10000
31177/31177 [==============================] - 1s 23us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 7/10000
31177/31177 [==============================] - 0s 13us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 8/10000
31177/31177 [==============================] - 0s 11us/sample - loss: 0.6460 - val_loss: 0.6369
Epoch 9/10000
31177/31177 [==============================] - 0s 9us/sample 

Epoch 74/10000
31177/31177 [==============================] - 0s 7us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 75/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 76/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 77/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 78/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 79/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 80/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 81/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 82/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6368
Epoch 83/1

31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 149/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 150/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 151/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 152/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 153/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 154/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 155/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 156/10000
31177/31177 [==============================] - 0s 6us/sample - loss: 0.6459 - val_loss: 0.6369
Epoch 157/10000
3

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),31177,0.016945,0.040503,0.692466,2.897286,-5.092881e-01,0.360054
Δ (t),31177,-0.016945,0.999485,-0.445748,47.576452,-2.951474e+01,24.394975
|Δ| (t),31177,0.645953,0.762881,5.957178,121.114726,1.060349e-05,29.514738
Δ² (t),31177,0.999225,7.035912,84.190951,9249.487395,1.124340e-10,871.119731
y^ (v),7929,0.016070,0.040001,0.756643,1.548500,-1.525939e-01,0.266285
Δ (v),7929,-0.005805,0.982582,-0.241873,15.063524,-1.201765e+01,12.887244
|Δ| (v),7929,0.636823,0.748268,3.970074,32.339555,2.449550e-05,12.887244
Δ² (v),7929,0.965378,3.988584,21.190011,683.354991,6.000295e-10,166.081057
y^ (*),39106,0.016768,0.040403,0.705300,2.634211,-5.092881e-01,0.360054
Δ (*),39106,-0.014686,0.996078,-0.406276,41.344616,-2.951474e+01,24.394975


In [ ]:
hist = DF(model.history.history)
hist.iloc[-110:-90]

In [ ]:
hist.iloc[-200:]

In [ ]:
w = model.get_weights(); w, [ l.shape for l in w ]

In [ ]:
describe(ty)